In [11]:
import cv2
import mediapipe as mp
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition


game_mode=0

#Capture camera
cap = cv2.VideoCapture(0)

#Call the gesture recognition model
mp_hands=mp.solutions.hands
mp_draw=mp.solutions.drawing_utils
gest = GestureRecognition(mode='eval')

#Finger Tracks
finger_tip_list=[]

# Load mediapipe hand class
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=2)

# Load display class
disp = DisplayHand(max_num_hands=2)

# Start video capture
cap = cv2.VideoCapture(0) # By default webcam is index 0

# Load gesture recognition class
gest = GestureRecognition(mode='eval')

counter = 0



cv2.namedWindow('gesture_recognition',cv2.WINDOW_NORMAL)


In [ ]:
while True:
    
    #Read frames
    ret,img=cap.read()
    
    if ret:
        
        #Image Flip
        img = cv2.flip(img, 1)
        
        img_hight=img.shape[0]
        img_width=img.shape[1]
        
        #Finger painting game
        if game_mode==0:
            #Image format conversion
            img_RGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
            #Hands landmarks identification
            results=mp_hands.Hands().process(img_RGB)
        
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_draw.draw_landmarks(img, hand_landmarks,mp_hands.HAND_CONNECTIONS)
                    for i,lm in enumerate(hand_landmarks.landmark):
                        if i==8:
                            finger_tip_list.append(lm)
                            if len(finger_tip_list)>100:
                                finger_tip_list.pop(0)
                            
                            
            #Plot fingertip trails                
            if finger_tip_list and game_mode==0:
                last_point=finger_tip_list[0].x*img_width,finger_tip_list[0].y*img_hight
                for point in finger_tip_list:
                    position=(point.x*img_width,point.y*img_hight)
        
                    cv2.line(img,(int(last_point[0]),int(last_point[1])),(int(position[0]),int(position[1])),color=(0, 255, 255), thickness=10)
                    last_point=position
            #Clear track    
            if cv2.waitKey(1) & 0xff == ord('c'):
                finger_tip_list=[]
            cv2.imshow('gesture_recognition', img)   
        
        
        
        #Rock Paper Scissors
        if game_mode==1:
            
            #improve performance
            img.flags.writeable = False

            # Feedforward to extract keypoint
            param = pipe.forward(img)
            # Evaluate gesture for all hands
            for p in param:
                if p['class'] is not None:
                    p['gesture'] = gest.eval(p['angle'])

            img.flags.writeable = True

            # Display keypoint and result of rock paper scissor game
            cv2.imshow('gesture_recognition', disp.draw_game_rps(img.copy(), param))
            
        
    if cv2.waitKey(1) & 0xff == ord('s'):
        game_mode=(game_mode+1)%2
    
    #Exit procedures
    if cv2.waitKey(1) & 0xff == ord('q'):
        cv2.destroyAllWindows()
        break
        
pipe.pipe.close()